# TNF Prediction: Baltimore Ravens at Miami Dolphins (Oct 30, 2025)

This notebook uses publicly available, **current** data to estimate the win probability for tonight's game and make a pick.

## Game
- **Matchup**: Baltimore Ravens @ Miami Dolphins  
- **Kick**: 8:15 PM ET (5:15 PM PT) on Oct 30, 2025  
- **Venue**: Hard Rock Stadium, Miami Gardens, FL

## Sources (fetched/verified by the assistant):
- **Schedule**: web sports schedule tool confirmed Ravens @ Dolphins on Oct 30, 2025 (TNF).
- **Odds** (widely reported consensus around the market): Ravens **-7.5** spread; total **51.5**; moneyline **Ravens -420 / Dolphins +330** (FOX Sports / BaltimoreBeatdown roundups).  
- **Injuries**: NFL.com Week 9 injury report + CBS Sports roundup indicate **Baltimore: no designations**; **Miami: 1 OUT, 1 DOUBTFUL, 4 QUESTIONABLE** as of this morning.
- **Weather**: Miami tonight: dry, mid-to-high 70s to ~80°F, breezy, low humidity (no heavy rain expected).

### Notes
- We treat the betting market as a **strong prior** (it incorporates a lot of information: team strength, injuries, travel/rest, etc.).  
- We then add small, transparent adjustments for **injury differential** and **home-field/weather** to get an ensemble view.


In [1]:
# Inputs captured from cited sources (timestamps in America/Denver)
from datetime import datetime, timezone
inputs = {
    'timestamp_local': datetime.now().isoformat(timespec='seconds'),
    'spread_favorite': -7.5,       # Ravens -7.5 (favorite negative)
    'total': 51.5,                 # O/U
    'moneyline_favorite': -420,    # Ravens -420
    'moneyline_underdog': 330,     # Dolphins +330
    'injuries_ravens': {'out': 0, 'doubtful': 0, 'questionable': 0},
    'injuries_dolphins': {'out': 1, 'doubtful': 1, 'questionable': 4},
    'weather': {
        'precip': False,
        'temp_f': 80,
        'breezy': True
    },
    'home_field': 'Dolphins'
}
inputs

{'timestamp_local': '2025-10-30T09:21:14',
 'spread_favorite': -7.5,
 'total': 51.5,
 'moneyline_favorite': -420,
 'moneyline_underdog': 330,
 'injuries_ravens': {'out': 0, 'doubtful': 0, 'questionable': 0},
 'injuries_dolphins': {'out': 1, 'doubtful': 1, 'questionable': 4},
 'weather': {'precip': False, 'temp_f': 80, 'breezy': True},
 'home_field': 'Dolphins'}

## 1) Market-implied probability
For a favorite with American odds **O\_fav < 0**, the implied win probability is $p = \frac{-O_{fav}}{-O_{fav} + 100}$.  
We'll also compute the underdog implied from its odds to sanity-check the market spread.

In [2]:
def implied_prob_from_american(odds):
    if odds < 0:
        return (-odds) / ((-odds) + 100)
    else:
        return 100 / (odds + 100)

p_ravens_market = implied_prob_from_american(inputs['moneyline_favorite'])
p_dolphins_market = implied_prob_from_american(inputs['moneyline_underdog'])
vig = p_ravens_market + p_dolphins_market - 1  # market vigorish approximation
p_ravens_market_fair = p_ravens_market / (1 + vig)
p_dolphins_market_fair = p_dolphins_market / (1 + vig)
p_ravens_market, p_dolphins_market, vig, p_ravens_market_fair, p_dolphins_market_fair

(0.8076923076923077,
 0.23255813953488372,
 0.0402504472271914,
 0.7764402407566638,
 0.2235597592433362)

## 2) Simple spread-to-win mapping (sanity check)
A crude logistic mapping from point spread to win prob often mirrors the market. We'll use a calibration so that **7.5** roughly maps to **~76%** and compare with the market implied.

In [4]:
import math

# Calibrate a simple logistic so that spread 0 -> 50%, spread 7.5 -> ~76%
def spread_to_prob(spread, k=0.18):
    # Larger k -> steeper curve; tuned to typical NFL relationships
    return 1/(1 + math.exp(-k*spread))

p_ravens_spread = spread_to_prob(abs(inputs['spread_favorite']))
p_ravens_spread

0.7941296281990525

## 3) Injury adjustment
We compute a very small penalty for the team with *more severe* injuries:  
- **Out = 1.0%**, **Doubtful = 0.6%**, **Questionable = 0.2%** off win probability.  
This is a transparent, conservative nudge layered on top of the market prior.

In [5]:
weights = {'out': 0.010, 'doubtful': 0.006, 'questionable': 0.002}

def injury_penalty(inj):
    return sum(weights[k]*inj.get(k,0) for k in weights)

pen_ravens = injury_penalty(inputs['injuries_ravens'])
pen_dolphins = injury_penalty(inputs['injuries_dolphins'])
net_shift_to_ravens = max(0.0, pen_dolphins - pen_ravens)
pen_ravens, pen_dolphins, net_shift_to_ravens

(0.0, 0.024, 0.024)

## 4) Weather/home-field adjustment
Weather is neutral-to-mild (dry, breezy, ~80°F). Miami has home field; we apply a **+1.5%** baseline to the home team unless severe weather pushes totals down.  
Because Baltimore is already a strong favorite and the weather is not adverse, the net **home bump** prevents overconfidence.

In [6]:
home_bump = 0.015  # given neutral weather
p_home = home_bump

# Ravens are away, so their probability gets reduced by the home bump
p_ravens_adj_market = max(0.0, min(1.0, (p_ravens_market_fair - p_home + net_shift_to_ravens)))
p_ravens_adj_spread = max(0.0, min(1.0, (p_ravens_spread      - p_home + net_shift_to_ravens)))
p_ravens_adj_market, p_ravens_adj_spread

(0.7854402407566639, 0.8031296281990525)

## 5) Ensemble aggregation
We average the **market** and the **spread-mapping** estimates after adjustments (equal weights).

In [7]:
p_ravens_final = 0.5*(p_ravens_adj_market + p_ravens_adj_spread)
p_dolphins_final = 1 - p_ravens_final
winner = 'Ravens' if p_ravens_final >= 0.5 else 'Dolphins'
p_ravens_final, p_dolphins_final, winner

(0.7942849344778582, 0.2057150655221418, 'Ravens')

## Result
- **Pick**: **Ravens**  
- **Estimated win probability**: the calculation above prints the numbers.  

### Interpretation (short)
- The market prior (moneyline -420) implies ~81% **before** removing vig and adjustments. After fairing the market and layering in injuries and home-field, we still sit ~mid-to-high 70s for the Ravens.  
- Miami's injury list (compared to Baltimore's clean sheet) and the lack of adverse weather point away from upset dynamics.  
- If you want to extend this, plug in team **EPA/play** offense/defense splits from RBSDM or SumerSports as numeric features and re-run a small logistic/GBM with prior = market.
